In [1]:
try:
    import mlxtend

    # Si mlxtend est déjà installé
    print("MLxtend est déjà installé (version {})".format(mlxtend.__version__))

except ImportError:
    # Si mlxtend n'est pas installé, installez-le
    !pip install mlxtend
    print("MLxtend a été installé avec succès.")

MLxtend est déjà installé (version 0.23.0)


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

## Installs

### Création du dataframe

In [ ]:
# Lecture du dataset
df = pd.read_csv("data/clean_dataset_googleplaystore.csv")
df.head(2)

In [16]:
def create_a_priori_df(df):
    # Création de notre dataframe respectant les conditions suivantes : 
    #TODO : Affiner les conditions avec peut-être d'autres résultats déjà trouvés
    # La colonne "Price" est à 1 si l'application est gratuite, 0 sinon.
    # La colonne "Rating" est à 1 si l'application a une note supérieure à 4.5, 0 sinon.
    # La colonne "Reviews" est à 1 si l'application a plus de 100000 reviews, 0 sinon.
    # La colonne "Installs" est à 1 si l'application a plus de 10000000 installs, 0 sinon.
    df_apriori = df.copy()
    df_apriori["Price"] = df_apriori["Price"].apply(lambda x: 1 if x == 0 else 0)
    df_apriori["Rating"] = df_apriori["Rating"].apply(lambda x: 1 if x > 4.5 else 0)
    df_apriori["Reviews"] = df_apriori["Reviews"].apply(lambda x: 1 if x > 100000 else 0)
    # df_apriori["Installs"] = df_apriori["Installs"].apply(lambda x: 1 if x > 10000000 else 0)

    # Ajoute les colonnes genre principal et genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre    
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)
    df_apriori = df_apriori[["App", "Price", "Rating", "Reviews", "GenrePrincipal", "GenreSecondaire", "Installs"]]


    # Sépare les genres principaux et secondaires en colonnes, chaque genre devient une colonne et le nom de cette colonne est la concaténation
    # de son nom et de 1 s'il est dans le genre principal ou 2 s'il est dans le genre secondaire. 
    # À pour valeur 1 si l'application est dans ce genre, 0 sinon.
    df_apriori["GenrePrincipal"] = df_apriori["GenrePrincipal"].astype(str) + "1"
    df_apriori["GenreSecondaire"] = df_apriori["GenreSecondaire"].astype(str) + "2"

    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)

    #Renommage de la colonne "nan2" en "NoSecondaryGenre"
    df_apriori = df_apriori.rename(columns={"nan2": "NoSecondaryGenre"})

    # Si une colonne à le type uint8, on la convertit en int64
    for column in df_apriori.columns:
        if df_apriori[column].dtype == "uint8":
            df_apriori[column] = df_apriori[column].astype("int64")

    df_apriori.info()

    # Enlever les lignes où installs n'est pas à 1
    # df_apriori = df_apriori[df_apriori["Installs"] == 1]

    return df_apriori

In [21]:
# Création de notre dataframe respectant les conditions suivantes : 
#TODO : Affiner les conditions avec peut-être d'autres résultats déjà trouvés
# La colonne "Price" est à 1 si l'application est gratuite, 0 sinon.
# La colonne "Rating" est à 1 si l'application a une note supérieure à 4.5, 0 sinon.
# La colonne "Reviews" est à 1 si l'application a plus de 100000 reviews, 0 sinon.
# La colonne "Installs" est à 1 si l'application a plus de 10000000 installs, 0 sinon.
df_apriori = df.copy()
df_apriori["Price"] = df_apriori["Price"].apply(lambda x: 1 if x == 0 else 0)
df_apriori["Rating"] = df_apriori["Rating"].apply(lambda x: 1 if x > 4.5 else 0)
df_apriori["Reviews"] = df_apriori["Reviews"].apply(lambda x: 1 if x > 100000 else 0)
# df_apriori["Installs"] = df_apriori["Installs"].apply(lambda x: 1 if x > 10000000 else 0)

# Ajoute les colonnes genre principal et genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre    
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)
df_apriori = df_apriori[["App", "Price", "Rating", "Reviews", "GenrePrincipal", "GenreSecondaire", "Installs"]]


# Sépare les genres principaux et secondaires en colonnes, chaque genre devient une colonne et le nom de cette colonne est la concaténation
# de son nom et de 1 s'il est dans le genre principal ou 2 s'il est dans le genre secondaire. 
# À pour valeur 1 si l'application est dans ce genre, 0 sinon.
df_apriori["GenrePrincipal"] = df_apriori["GenrePrincipal"].astype(str) + "1"
df_apriori["GenreSecondaire"] = df_apriori["GenreSecondaire"].astype(str) + "2"

df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)

#Renommage de la colonne "nan2" en "NoSecondaryGenre"
df_apriori = df_apriori.rename(columns={"nan2": "NoSecondaryGenre"})

# Enlever les lignes où installs n'est pas à 1
# df_apriori = df_apriori[df_apriori["Installs"] == 1]

df_apriori.head(2)

,App,Price,Rating,Reviews,GenrePrincipal,GenreSecondaire,Installs,Action1,Adventure1,Arcade1,...,Video Players & Editors1,Weather1,Word1,Action & Adventure2,Brain Games2,Creativity2,Education2,Music & Video2,Pretend Play2,NoSecondaryGenre
0,Photo Editor & Candy Camera & Grid & ScrapBook,1,0,0,Art & Design1,nan2,10000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Coloring book moana,1,0,0,Art & Design1,Pretend Play2,500000,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Pivot

In [22]:
# Exclure des colonnes
excluded_columns = ["App", "Installs", "Reviews", "GenrePrincipal", "GenreSecondaire"]
pivoted = df_apriori.pivot_table(index='App', columns='Installs', values=[col for col in df_apriori.columns if col not in excluded_columns], aggfunc='mean')


# Afficher la liste des colonnes sélectionnées
pivoted = pivoted.fillna(0)
pivoted = pivoted.astype(int)

pivoted.head()

Action & Adventure2             \
Installs                                      0          1            
App                                                                   
"i DT" Fútbol. Todos Somos Técnicos.                   0          0   
+Download 4 Instagram Twitter                          0          0   
- Free Comics - Comic Apps                             0          0   
.R                                                     0          0   
/u/app                                                 0          0   

                                                                       \
Installs                             5          10         50           
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       \
Installs                             100        500        1000         
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                            ...      Word1  \
Installs                             5000       10000       ... 50000        
App                                                         ...              
"i DT" Fútbol. Todos Somos Técnicos.          0          0  ...          0   
+Download 4 Instagram Twitter                 0          0  ...          0   
- Free Comics - Comic Apps                    0          0  ...          0   
.R                                            0          0  ...          0   
/u/app                                        0          0  ...          0   

                                                                       \
Installs                             100000     500000     1000000      
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       \
Installs                             5000000    10000000   50000000     
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       
Installs                             100000000  500000000  1000000000  
App                                                                    
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0  
+Download 4 Instagram Twitter                 0          0          0  
- Free Comics - Comic Apps                    0          0          0  
.R                                            0          0          0  

### Support

In [23]:
supports = apriori(pivoted, min_support=0.005,max_len=3, use_colnames=True)
supports.sort_values(by="support", ascending=False).head()

d:\Programmes\Python\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
51,0.137657,"((Price, 1000000))"
31,0.128527,"((NoSecondaryGenre, 1000000))"
141,0.126630,"((Price, 1000000), (NoSecondaryGenre, 1000000))"
29,0.114062,"((NoSecondaryGenre, 100000))"
27,0.114062,"((NoSecondaryGenre, 10000))"


## Installs + Reviews

In [29]:
df2 = pd.read_csv("data/clean_dataset_googleplaystore.csv")

In [30]:
# Création de notre dataframe respectant les conditions suivantes : 
#TODO : Affiner les conditions avec peut-être d'autres résultats déjà trouvés
# La colonne "Price" est à 1 si l'application est gratuite, 0 sinon.
# La colonne "Rating" est à 1 si l'application a une note supérieure à 4.5, 0 sinon.
# La colonne "Reviews" est à 1 si l'application a plus de 100000 reviews, 0 sinon.
# La colonne "Installs" est à 1 si l'application a plus de 10000000 installs, 0 sinon.
df_apriori = df2.copy()
df_apriori["Price"] = df_apriori["Price"].apply(lambda x: 1 if x == 0 else 0)
df_apriori["Rating"] = df_apriori["Rating"].apply(lambda x: 1 if x > 4.5 else 0)
# df_apriori["Installs"] = df_apriori["Installs"].apply(lambda x: 1 if x > 10000000 else 0)

# Ajoute les colonnes genre principal et genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre    
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)
df_apriori = df_apriori[["App", "Price", "Rating", "Reviews", "GenrePrincipal", "GenreSecondaire", "Installs"]]


# Sépare les genres principaux et secondaires en colonnes, chaque genre devient une colonne et le nom de cette colonne est la concaténation
# de son nom et de 1 s'il est dans le genre principal ou 2 s'il est dans le genre secondaire. 
# À pour valeur 1 si l'application est dans ce genre, 0 sinon.
df_apriori["GenrePrincipal"] = df_apriori["GenrePrincipal"].astype(str) + "1"
df_apriori["GenreSecondaire"] = df_apriori["GenreSecondaire"].astype(str) + "2"

df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)

#Renommage de la colonne "nan2" en "NoSecondaryGenre"
df_apriori = df_apriori.rename(columns={"nan2": "NoSecondaryGenre"})

# Enlever les lignes où installs n'est pas à 1
# df_apriori = df_apriori[df_apriori["Installs"] == 1]

df_apriori.head(2)

,App,Price,Rating,Reviews,GenrePrincipal,GenreSecondaire,Installs,Action1,Adventure1,Arcade1,...,Video Players & Editors1,Weather1,Word1,Action & Adventure2,Brain Games2,Creativity2,Education2,Music & Video2,Pretend Play2,NoSecondaryGenre
0,Photo Editor & Candy Camera & Grid & ScrapBook,1,0,159,Art & Design1,nan2,10000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Coloring book moana,1,0,967,Art & Design1,Pretend Play2,500000,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
# Exclure des colonnes
excluded_columns = ["App", "Installs", "Reviews", "GenrePrincipal", "GenreSecondaire"]
pivoted = df_apriori.pivot_table(index='App', columns='Installs', values=[col for col in df_apriori.columns if col not in excluded_columns], aggfunc='mean')

# Afficher la liste des colonnes sélectionnées
pivoted = pivoted.fillna(0)
pivoted = pivoted.astype(int)

pivoted.head()

Action & Adventure2             \
Installs                                      0          1            
App                                                                   
"i DT" Fútbol. Todos Somos Técnicos.                   0          0   
+Download 4 Instagram Twitter                          0          0   
- Free Comics - Comic Apps                             0          0   
.R                                                     0          0   
/u/app                                                 0          0   

                                                                       \
Installs                             5          10         50           
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       \
Installs                             100        500        1000         
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                            ...      Word1  \
Installs                             5000       10000       ... 50000        
App                                                         ...              
"i DT" Fútbol. Todos Somos Técnicos.          0          0  ...          0   
+Download 4 Instagram Twitter                 0          0  ...          0   
- Free Comics - Comic Apps                    0          0  ...          0   
.R                                            0          0  ...          0   
/u/app                                        0          0  ...          0   

                                                                       \
Installs                             100000     500000     1000000      
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       \
Installs                             5000000    10000000   50000000     
App                                                                     
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0   
+Download 4 Instagram Twitter                 0          0          0   
- Free Comics - Comic Apps                    0          0          0   
.R                                            0          0          0   
/u/app                                        0          0          0   

                                                                       
Installs                             100000000  500000000  1000000000  
App                                                                    
"i DT" Fútbol. Todos Somos Técnicos.          0          0          0  
+Download 4 Instagram Twitter                 0          0          0  
- Free Comics - Comic Apps                    0          0          0  
.R                                            0          0          0  

In [35]:
supports = apriori(pivoted, min_support=0.005,max_len=3, use_colnames=True)

d:\Programmes\Python\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [36]:
supports.sort_values(by="support", ascending=False).head()

,support,itemsets
51,0.137657,"((Price, 1000000))"
31,0.128527,"((NoSecondaryGenre, 1000000))"
141,0.126630,"((Price, 1000000), (NoSecondaryGenre, 1000000))"
29,0.114062,"((NoSecondaryGenre, 100000))"
27,0.114062,"((NoSecondaryGenre, 10000))"
